In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from darts.datasets import ETTh1Dataset
from darts.models import NLinearModel
from darts.metrics.metrics import mae, mse
import numpy as np
import pandas as pd
import random
import csv
import os, subprocess, gc, time, datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR
from torchmetrics.regression import MeanAbsoluteError, MeanSquaredError
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from itertools import product

import optuna
from optuna.trial import TrialState
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_timeline

import models.original_models as original_models
import models.models_3d_atomics_on_variate_to_concepts as new_models
from vasopressor.preprocess_helpers import *
from models.helper import *
from models.param_initializations import *
from models.optimization_strategy import greedy_selection
from models.TimeSeriesDataset import TimeSeriesDataset

gpu_id = int(subprocess.check_output('nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader | nl -v 0 | sort -nrk 2 | cut -f 1 | head -n 1 | xargs', shell=True, text=True))
device = torch.device(f'cuda:{gpu_id}') if torch.cuda.is_available else torch.device('cpu')
print("current device", device)

current device cuda:8


In [2]:
series = ETTh1Dataset().load()

print(series.start_time())
print(series.end_time())

# series.plot()

2016-07-01 00:00:00
2018-06-26 19:00:00


In [3]:
train_series, test_series = series.split_before(0.6)
val_series, test_series = test_series.split_before(0.5)


In [4]:
def preprocess_data(series, seq_len, window_stride=1, pred_len=1, batch_size = 512):
    scaler = StandardScaler()
    
    train, test = series.split_before(0.6)
    val, test = test.split_before(0.5)
    
    print("Train/Val/Test", len(train), len(val), len(test))
    
    train_og = train.pd_dataframe()
    train = scaler.fit_transform(train_og)
    train = pd.DataFrame(train, columns=train_og.columns)
    X_train = train
    y_train = train # [["OT"]]
    X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
    y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32)
    
    indicators = torch.isfinite(X_train)
    X_train = torch.cat([X_train, indicators], axis=1)
    
    train_dataset = TimeSeriesDataset(X_train, y_train, seq_len, window_stride, pred_len)
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False, num_workers=4, pin_memory=True)

    val_og = val.pd_dataframe()
    val = scaler.transform(val_og)
    val = pd.DataFrame(val, columns=val_og.columns)
    X_val = val
    y_val = val # [["OT"]]
    X_val = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
    y_val = torch.tensor(y_val.to_numpy(), dtype=torch.float32)
    
    indicators = torch.isfinite(X_val)
    X_val = torch.cat([X_val, indicators], axis=1)
    
    val_dataset = TimeSeriesDataset(X_val, y_val, seq_len, window_stride, pred_len)
    val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, num_workers=4, pin_memory=True)

    test_og = test.pd_dataframe()
    test = scaler.transform(test_og)
    test = pd.DataFrame(test, columns=test_og.columns)
    X_test = test
    y_test = test # [["OT"]]
    X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
    y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)
    
    indicators = torch.isfinite(X_test)
    X_test = torch.cat([X_test, indicators], axis=1)
    
    test_dataset = TimeSeriesDataset(X_test, y_test, seq_len, window_stride, pred_len)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    return train_loader, val_loader, test_loader, scaler


In [5]:
seq_len = 10
train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=1)

for X,y in train_loader:
    print(X.shape)
    print(y.shape)
    break

print("Batches", len(train_loader), len(val_loader), len(test_loader))

Train/Val/Test 10451 3484 3485
torch.Size([512, 10, 14])
torch.Size([512, 7])
Batches 21 7 7


In [6]:
# plots
def plot_losses(train_losses, val_losses):
    plt.plot(train_losses, color="black", label="Train")
    plt.plot(val_losses, color="green", label="Val")
    plt.legend()
    plt.show()

def plot_mae_mse(history, title, dec="{:.3g}"):
    xticks = range(len(history[:, 0]))
    plt.plot(xticks, history[:, 2], label='MAE')
    plt.plot(xticks, history[:, 3], label='MSE')

    plt.xlabel('Num Concepts')
    plt.ylabel('Criteria')
    # plt.ylim(0, 1)
    xtick_labels = list(map(int, history[:, 0]))
    plt.xticks(xticks, xtick_labels)
    plt.yscale('log')

    if dec:
        for x,_y in zip(xticks, history[:, 2]):
            label = dec.format(_y)
            plt.annotate(label, # this is the text
                        (x,_y), # these are the coordinates to position the label
                        textcoords="offset points", # how to position the text
                        xytext=(0,-10), # distance from text to points (x,y)
                        ha='center') # horizontal alignment can be left, right or center
            
        for x,_y in zip(xticks, history[:, 3]):
            label = dec.format(_y)
            plt.annotate(label, # this is the text
                        (x,_y), # these are the coordinates to position the label
                        textcoords="offset points", # how to position the text
                        xytext=(0,-10), # distance from text to points (x,y)
                        ha='center') # horizontal alignment can be left, right or center

    plt.title(title)
    plt.legend()
    plt.show()


def plot_atomics_concepts_metric(history, title, dec="{:.3g}"):
        
    df = pd.DataFrame(history, columns=["n_atomics", "n_concepts", "val_loss", "mae", "mse"])
    mean_atomics = df.groupby("n_atomics").mean()
    mean_concepts = df.groupby("n_concepts").mean()

    # display(mean_atomics)
    plt.plot(mean_atomics.index, mean_atomics["mae"], label='MAE')
    plt.plot(mean_atomics.index, mean_atomics["mse"], label='MSE')
    plt.xlabel('Num Atomics')
    plt.ylabel('Criteria')
    plt.title("Metric as mean over atomics")
    plt.suptitle(title)
    plt.legend()
    plt.show()

    # display(mean_concepts)
    plt.plot(mean_concepts.index, mean_concepts["mae"], label='MAE')
    plt.plot(mean_concepts.index, mean_concepts["mse"], label='MSE')
    plt.xlabel('Num Concepts')
    plt.ylabel('Criteria')
    plt.title("Metric as mean over concepts")
    plt.suptitle(title)
    plt.legend()
    plt.show()
    

def plot_prediction_vs_true(yb, preds, title):
    plt.plot(yb, color="black", label="True")
    plt.plot(preds, color="red", label="Pred")
    plt.title(title)
    plt.legend()
    plt.show()


In [7]:
def initializeModel(n_concepts, input_dim, changing_dim, seq_len, output_dim, 
                    use_indicators, use_fixes, use_only_last_timestep, top_k=''):
    model = original_models.CBM(input_dim = input_dim, 
                                changing_dim = changing_dim, 
                                seq_len = seq_len,
                                num_concepts = n_concepts,
                                use_indicators = use_indicators,
                                use_fixes = use_fixes,
                                use_only_last_timestep = use_only_last_timestep,
                                use_grad_norm = False,
                                opt_lr = 1e-3,
                                opt_weight_decay = 1e-5,
                                l1_lambda=1e-3,
                                cos_sim_lambda=1e-2,
                                output_dim = output_dim,
                                top_k=top_k,
                                device = device,
                                task_type=original_models.TaskType.REGRESSION,
                                )
    model = model.to(device)
    return model

def initializeModel_with_atomics(n_atomics, n_concepts, input_dim, changing_dim, seq_len, output_dim, 
                                 use_summaries_for_atomics, use_indicators, use_fixes, top_k=''):
    model = new_models.CBM(input_dim = input_dim, 
                            changing_dim = changing_dim, 
                            seq_len = seq_len,
                            num_concepts = n_concepts,
                            num_atomics= n_atomics,
                            use_summaries_for_atomics = use_summaries_for_atomics,
                            use_indicators = use_indicators,
                            use_fixes = use_fixes,
                            use_grad_norm = False,
                            opt_lr = 1e-3,
                            opt_weight_decay = 1e-5,
                            l1_lambda=1e-3,
                            cos_sim_lambda=1e-2,
                            output_dim = output_dim,
                            top_k=top_k,
                            device = device,
                            task_type=new_models.TaskType.REGRESSION,
                            )
    model = model.to(device)
    return model


In [8]:
random_seed = 1
set_seed(random_seed)


In [9]:
seq_len = 336
pred_len = 96
n_atomics_list = list(range(2,11,2))
n_concepts_list = list(range(2,11,2))
changing_dim = len(series.columns)
input_dim = 2 * changing_dim


In [10]:
mae_metric = MeanAbsoluteError().to(device)
mse_metric = MeanSquaredError().to(device)


## Original

In [11]:
config_original = {
    "n_concepts": [4, 20],
    "use_indicators": [True, False],
    "use_fixes": [False, True],
    "use_only_last_timestep": [True, False],
}

all_config_permutations_og = list(product(*config_original.values()))
all_config_permutations_og = [dict(zip(config_original.keys(), permutation)) for permutation in all_config_permutations_og]
print(len(all_config_permutations_og))
# all_config_permutations_og

16


In [12]:
experiment_folder = f"/workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L{seq_len}-T{pred_len}/"
model_path_og = experiment_folder + "forecasting_c{n_concepts}_ind{use_indicators}_fixes{use_fixes}_onlylasttimestep{use_only_last_timestep}.pt"

if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)

In [13]:
history_original = []


for i, config in enumerate(all_config_permutations_og):
    print(i, config)
    
    train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=1)

    model = initializeModel(**config, input_dim=input_dim, changing_dim=changing_dim, seq_len=seq_len, output_dim=changing_dim)
    model.fit(train_loader, val_loader, p_weight=None, save_model_path=model_path_og.format(**config), max_epochs=10000)
    
    # display(model)
    
    train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=pred_len)
    
    model.eval()
    with torch.inference_mode():
        for batch_idx, (Xb, yb) in enumerate(test_loader):
            Xb, yb = Xb.to(device), yb.to(device)
            preds = model.forward_one_step_ahead(Xb, steps_ahead=pred_len)
            
            mae = mae_metric(preds, yb).item()
            mse = mse_metric(preds, yb).item()
        mae = mae_metric.compute().item()
        mse = mse_metric.compute().item()
        mae_metric.reset()
        mse_metric.reset()
    
    history = ["original", i, model.val_losses[-1], mae, mse]
    display(history)
    history_original.append(np.array(history))
    
    # plot_losses(model.train_losses, model.val_losses)
    
history_original = np.array(history_original)
history_original.shape


0 {'n_concepts': 4, 'use_indicators': True, 'use_fixes': False, 'use_only_last_timestep': True}
Train/Val/Test 10451 3484 3485


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Loaded model from /workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L336-T96/forecasting_c4_indTrue_fixesFalse_onlylasttimestepTrue.pt
Train/Val/Test 10451 3484 3485


['original', 0, 0.2691112756729126, 1.1771647930145264, 2.7613794803619385]

1 {'n_concepts': 4, 'use_indicators': True, 'use_fixes': False, 'use_only_last_timestep': False}
Train/Val/Test 10451 3484 3485
Loaded model from /workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L336-T96/forecasting_c4_indTrue_fixesFalse_onlylasttimestepFalse.pt


...

15 {'n_concepts': 20, 'use_indicators': False, 'use_fixes': True, 'use_only_last_timestep': False}
Train/Val/Test 10451 3484 3485
Loaded model from /workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L336-T96/forecasting_c20_indFalse_fixesTrue_onlylasttimestepFalse.pt


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Train/Val/Test 10451 3484 3485


['original', 15, 0.8202223181724548, 0.7474615573883057, 0.8774258494377136]

(16, 5)

In [14]:
# plot_mae_mse(history_original, "Original")


In [15]:
# # Plot Prediction vs actual
# train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=pred_len)

# mae_metric = MeanAbsoluteError().to(device)
# mse_metric = MeanSquaredError().to(device)
# n_concepts = 10

# model = initializeModel(n_concepts, input_dim, changing_dim, seq_len, output_dim=pred_len)
# model.fit(train_loader, val_loader, None, save_model_path=model_path_og.format(n_concepts), max_epochs=10000)

# model.eval()
# with torch.no_grad():
#     for batch_idx, (Xb, yb) in enumerate(val_loader):
#         Xb, yb = Xb.to(device), yb.to(device)
#         preds = model.forward(Xb)
        
#         mae = mae_metric(preds, yb).item()
#         mse = mse_metric(preds, yb).item()
#         break
#     mae = mae_metric.compute().item()
#     mse = mse_metric.compute().item()
#     mae_metric.reset()
#     mse_metric.reset()


# i = 20
# yb = yb.cpu().numpy()[i]
# preds = preds.cpu().numpy()[i]

# print(yb.shape)
# print(preds.shape)

# plot_prediction_vs_true(yb, preds, title=f"Original - Predictions with {n_concepts} Concepts")


## Atomics

In [16]:
config_atomics = {
    "n_atomics": [10, 30, 100],
    "n_concepts": [4, 20],
    "use_indicators": [True, False],
    "use_fixes": [False, True],
    "use_summaries_for_atomics": [True, False],
}

all_config_permutations_atomics = list(product(*config_atomics.values()))
all_config_permutations_atomics = [dict(zip(config_atomics.keys(), permutation)) for permutation in all_config_permutations_atomics]
print(len(all_config_permutations_atomics))
# all_config_permutations_atomics

48


In [17]:
experiment_folder = f"/workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L{seq_len}-T{pred_len}/"
model_path_re = experiment_folder + "forecasting_a{n_atomics}_c{n_concepts}_ind{use_indicators}_fixes{use_fixes}_summaries{use_summaries_for_atomics}.pt"

if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)

In [18]:
history_atomics = []


for i, config in enumerate(all_config_permutations_atomics):
    print(i, config)
    
    train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=1)
    
    model = initializeModel_with_atomics(**config, input_dim=input_dim, changing_dim=changing_dim, seq_len=seq_len, output_dim=changing_dim)
    model.fit(train_loader, val_loader, p_weight=None, save_model_path=model_path_re.format(**config), max_epochs=10000)
    
    # display(model)
    
    train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=pred_len)
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (Xb, yb) in enumerate(test_loader):
            Xb, yb = Xb.to(device), yb.to(device)
            preds = model.forward_one_step_ahead(Xb, steps_ahead=pred_len)
            
            mae = mae_metric(preds, yb).item()
            mse = mse_metric(preds, yb).item()
        mae = mae_metric.compute().item()
        mse = mse_metric.compute().item()
        mae_metric.reset()
        mse_metric.reset()
    
    history = ["atomics", i, model.val_losses[-1], mae, mse]
    display(history)
    history_atomics.append(np.array(history))

    # plot_losses(model.train_losses, model.val_losses)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
history_atomics = np.array(history_atomics)
history_atomics.shape


0 {'n_atomics': 10, 'n_concepts': 4, 'use_indicators': True, 'use_fixes': False, 'use_summaries_for_atomics': True}
Train/Val/Test 10451 3484 3485
Loaded model from /workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L336-T96/forecasting_a10_c4_indTrue_fixesFalse_summariesTrue.pt


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Train/Val/Test 10451 3484 3485


['atomics', 0, 0.49742478132247925, 0.9970691204071045, 1.6400070190429688]

1 {'n_atomics': 10, 'n_concepts': 4, 'use_indicators': True, 'use_fixes': False, 'use_summaries_for_atomics': False}
Train/Val/Test 10451 3484 3485
Loaded model from /workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L336-T96/forecasting_a10_c4_indTrue_fixesFalse_summariesFalse.pt


...

47 {'n_atomics': 100, 'n_concepts': 20, 'use_indicators': False, 'use_fixes': True, 'use_summaries_for_atomics': False}
Train/Val/Test 10451 3484 3485
Loaded model from /workdir/optimal-summaries-public/_models/etth1/1step/ablaition-L336-T96/forecasting_a100_c20_indFalse_fixesTrue_summariesFalse.pt


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Train/Val/Test 10451 3484 3485


['atomics', 47, 0.2009379267692566, 0.8505852222442627, 1.2636874914169312]

(48, 5)

In [25]:
columns=["type", "config", "val_loss", "mae", "mse"]
dtypes = {'type': str, 'config': int, 'val_loss': float, 'mae': float, 'mse': float}

df_og = pd.DataFrame(history_original, columns=columns).astype(dtypes)
df_og = pd.concat([df_og, pd.DataFrame(all_config_permutations_og)], axis=1)

df_atomics = pd.DataFrame(history_atomics, columns=columns).astype(dtypes)
df_atomics = pd.concat([df_atomics, pd.DataFrame(all_config_permutations_atomics)], axis=1)

result_df = pd.concat([df_og, df_atomics], ignore_index=True)
# result_df

In [26]:
for col in result_df.columns[3:6]:
    baseline = result_df[(result_df['type'] == 'original') & (result_df['config'] == 0)][col].values[0]
    print(col, baseline)
    result_df[f'{col}_abs_imp'] = result_df[col] - baseline
    # result_df[f'{col}_rel_imp'] = result_df[f'{col}_abs_imp'] / baseline
# result_df

mae 1.1771647930145264
mse 2.7613794803619385
n_concepts 4


In [27]:
pd.set_option('display.max_rows', 100)
result_df.sort_values(by='mse', ascending=True)
# atomics: atomics, concepts, use_indicators, use_fixes, output_dim, use_summaries_for_atomics
# original: concepts, use_indicators, use_fixes, output_dim, use_only_last_timestep


,type,config,val_loss,mae,mse,n_concepts,use_indicators,use_fixes,use_only_last_timestep,n_atomics,use_summaries_for_atomics,mae_abs_imp,mse_abs_imp,n_concepts_abs_imp
11,original,11,0.789226,0.690913,0.788829,20,True,True,False,NaN,NaN,-0.486251,-1.972550,16
9,original,9,0.715416,0.692953,0.807579,20,True,False,False,NaN,NaN,-0.484212,-1.953801,16
13,original,13,0.640830,0.692244,0.854890,20,False,False,False,NaN,NaN,-0.484921,-1.906490,16
15,original,15,0.820222,0.747462,0.877426,20,False,True,False,NaN,NaN,-0.429703,-1.883954,16
53,atomics,37,0.469095,0.749761,0.934202,4,False,False,NaN,100.0,False,-0.427404,-1.827178,0
3,original,3,0.863633,0.780468,1.003076,4,True,True,False,NaN,NaN,-0.396697,-1.758303,0
58,atomics,42,0.258789,0.749811,1.017030,20,True,True,NaN,100.0,True,-0.427354,-1.744349,16
7,original,7,0.839636,0.741249,1.020023,4,False,True,False,NaN,NaN,-0.435916,-1.741356,0
38,atomics,22,0.451580,0.784717,1.043705,4,False,True,NaN,30.0,True,-0.392448,-1.717674,0
28,atomics,12,0.178505,0.749474,1.047484,20,False,False,NaN,10.0,True,-0.427691,-1.713896,16


In [28]:
for key in set(list(all_config_permutations_og[0].keys()) + list(all_config_permutations_atomics[0].keys())):
    display(result_df.groupby(key)[["mae", "mse"]].mean())

display(result_df.groupby("type")[["mae", "mse"]].mean())

,mae,mse
n_atomics,,
10.0,0.920387,1.427469
30.0,0.945309,1.519319
100.0,0.918190,1.460378


,mae,mse
n_concepts,,
4,0.904763,1.413003
20,0.908834,1.417645


,mae,mse
use_indicators,,
False,0.882083,1.322922
True,0.931514,1.507726


,mae,mse
use_fixes,,
False,0.892319,1.378921
True,0.921279,1.451727


,mae,mse
use_summaries_for_atomics,,
False,0.880697,1.327004
True,0.975227,1.611106


,mae,mse
use_only_last_timestep,,
False,0.751296,0.969896
True,0.935321,1.538364


,mae,mse
type,,
atomics,0.927962,1.469055
original,0.843308,1.254130


In [ ]:
# plot_mae_mse(history_re, "Atomics cat Summaries")
# plot_atomics_concepts_metric(history_atomics, "Atomics cat Summaries")


In [ ]:
# # Plot Prediction vs actual
# train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=pred_len)

# mae_metric = MeanAbsoluteError().to(device)
# mse_metric = MeanSquaredError().to(device)
# n_concepts = 600

# model = initializeModel_redesigned(n_concepts, input_dim, changing_dim, seq_len, output_dim=pred_len)
# model.fit(train_loader, val_loader, None, save_model_path=model_path_re.format(n_concepts), max_epochs=10000)

# model.eval()
# with torch.no_grad():
#     for batch_idx, (Xb, yb) in enumerate(val_loader):
#         Xb, yb = Xb.to(device), yb.to(device)
#         preds = model.forward(Xb)
        
#         mae = mae_metric(preds, yb).item()
#         mse = mse_metric(preds, yb).item()
#         break
#     mae = mae_metric.compute().item()
#     mse = mse_metric.compute().item()
#     mae_metric.reset()
#     mse_metric.reset()


# i = 20
# yb = yb.cpu().numpy()[i]
# preds = preds.cpu().numpy()[i]

# print(yb.shape)
# print(preds.shape)

# plot_prediction_vs_true(yb, preds, title=f"Redesigned - Predictions with {n_concepts} Concepts")


## Input cat Summaries to Atomics

In [ ]:
experiment_folder = f"/workdir/optimal-summaries-public/_models/etth1/input-cat-summaries-L{seq_len}-T{pred_len}/"
model_path_re = experiment_folder + "forecasting_c{}_a{}.pt"

if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)

In [ ]:
history_atomics = []

train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=pred_len)

mae_metric = MeanAbsoluteError().to(device)
mse_metric = MeanSquaredError().to(device)


for n_concepts in n_concepts_list:
    for n_atomics in n_atomics_list:
        print("n_atomics", n_atomics, "n_concepts", n_concepts)
        
        model = initializeModel_with_atomics(n_atomics, n_concepts, input_dim, changing_dim, seq_len, output_dim=pred_len, use_summaries_for_atomics=True)
        print("test", model.output_dim)
        model.fit(train_loader, val_loader, None, save_model_path=model_path_re.format(n_concepts, n_atomics), max_epochs=10000)
        
        print("Trained for ", model.curr_epoch+1)
        display(model)
        
        model.eval()
        with torch.no_grad():
            for batch_idx, (Xb, yb) in enumerate(test_loader):
                Xb, yb = Xb.to(device), yb.to(device)
                preds = model.forward(Xb)
                
                mae = mae_metric(preds, yb).item()
                mse = mse_metric(preds, yb).item()
            mae = mae_metric.compute().item()
            mse = mse_metric.compute().item()
            mae_metric.reset()
            mse_metric.reset()
        
        history = [n_atomics, n_concepts, round(model.val_losses[-1],2), mae, mse]
        display(history)
        history_atomics.append(np.array(history))
    
        plot_losses(model.train_losses, model.val_losses)
        torch.cuda.empty_cache()
    
history_atomics = np.array(history_atomics)
history_atomics.shape


In [ ]:
# plot_mae_mse(history_re, "Input cat Summaries")
plot_atomics_concepts_metric(history_atomics, "Input cat Summaries")


In [ ]:
# Plot Prediction vs actual
train_loader, val_loader, test_loader, scaler = preprocess_data(series, seq_len, pred_len=pred_len)

mae_metric = MeanAbsoluteError().to(device)
mse_metric = MeanSquaredError().to(device)

n_concepts = 47
n_atomics = 34

model = initializeModel_with_atomics(n_atomics, n_concepts, input_dim, changing_dim, seq_len, output_dim=pred_len, use_summaries_for_atomics=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=model.optimizer, patience=5) 

model.fit(train_loader, val_loader, None, save_model_path=model_path_re.format(n_concepts, n_atomics), max_epochs=10000, scheduler=scheduler)


model.eval()
with torch.no_grad():
    for batch_idx, (Xb, yb) in enumerate(val_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model.forward(Xb)
        
        mae = mae_metric(preds, yb).item()
        mse = mse_metric(preds, yb).item()
    mae = mae_metric.compute().item()
    mse = mse_metric.compute().item()
    mae_metric.reset()
    mse_metric.reset()

    print("MSE", mse, "MAE", mae)

    for batch_idx, (Xb, yb) in enumerate(train_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        preds = model(Xb)
        break

i = 20
yb = yb.cpu().numpy()[i]
preds = preds.cpu().numpy()[i]

print(yb.shape)
print(preds.shape)

plot_prediction_vs_true(yb, preds, title=f"Redesigned - Predictions with {n_concepts} Concepts")


In [ ]:

plt.plot(model.val_losses)
plt.plot(model.train_losses, label="Train")
plt.legend()
plt.show()

## Optimization

In [ ]:
# feature weights
n_concepts = 5

model = initializeModel(n_concepts, input_dim, changing_dim, seq_len)
model.fit(train_loader, val_loader, None, model_path.format(n_concepts), 1000)

for name, param in model.named_parameters():
    if "bottleneck.weight" in name:
        bottleneck_weights = param
feature_weights = bottleneck_weights.cpu().detach().numpy()

feature_weights.shape

In [ ]:
# visualize weight magnitudes
for c in range(n_concepts):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    inds = np.argsort(-np.abs(feature_weights[c]))[:100]
    ax.bar(np.arange(1,101),np.abs(feature_weights[c])[inds])
    ax.set_xlabel("Top 100 features")
    ax.set_ylabel("abs value of feature coefficient")
    plt.show()


In [ ]:
# get 90th percentile of feature weights
sum90p = np.sum(np.abs(feature_weights), axis=-1)*0.90
sum90p.shape


In [ ]:
# get top K indizes
top_k_inds = []
for c in range(n_concepts):
    topkinds_conc = []
    curr_sum = 0
    inds = np.argsort(-np.abs(feature_weights[c])) #desc
    sorted_weights = feature_weights[c][inds]
    
    for ind, weight in zip(inds, sorted_weights):
        curr_sum += abs(weight)
        if curr_sum <= sum90p[c]:
            topkinds_conc.append(ind)
        else:
            break
    
    # if selects less than 10, choose 10 best
    if len(topkinds_conc) < 10:
        topkinds_conc = np.argsort(-np.abs(feature_weights[c]))[:10].tolist()
    
    top_k_inds.append(topkinds_conc)

top_k_inds

In [ ]:
# write top k inds to csv
filename = experiment_folder + "top-k/top_k_inds_c{}.csv".format(n_concepts)

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)
    # writing the data rows 
    csvwriter.writerows(top_k_inds)


In [ ]:
best_aucs, best_auc_inds, best_auc_concepts = greedy_selection(auroc_metric, test_loader, top_k_inds, model)


In [ ]:
filename = experiment_folder + "top-k/bottleneck_r{}_c{}_topkinds.csv".format(random_seed, n_concepts)

# writing to csv file
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Best AUC", "Best AUC Concept #", "Best AUC ind #"])
    # writing the data rows 
    for row in zip(best_aucs, best_auc_concepts, best_auc_inds):
        csvwriter.writerow(list(row))


## Multi-class

In [ ]:
experiment_folder = "/workdir/optimal-summaries-public/_models/arabic/multiclass/"
model_path = experiment_folder + "arabic_c{}.pt"
random_seed = 1

if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)

In [ ]:
history_multiclass = []

set_seed(random_seed)

data, y_ohe, num_classes, weights = preprocess_data_multiclass(X, y)
train_loader, val_loader, test_loader = initialize_data(1, data, y_ohe, multiclass=True)

input_dim = data.shape[2]
changing_dim = X[0].shape[0]
seq_len = data.shape[1]

auroc_metric = AUROC(task="multiclass", num_classes=num_classes).to(device)
accuracy_metric = Accuracy(task="multiclass", num_classes=num_classes).to(device)

for n_concepts in range(1,16):
    print(n_concepts)
    
    model = initializeModel(n_concepts, input_dim, changing_dim, seq_len, num_classes)
    model.fit(train_loader, val_loader, weights, model_path.format(n_concepts), 1000)
    
    for batch_idx, (Xb, yb) in enumerate(test_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        probs = model.forward_probabilities(Xb)
        
        auc = auroc_metric(probs, yb).item()
        acc = accuracy_metric(probs, yb).item()
    auc = auroc_metric.compute().item()
    acc = accuracy_metric.compute().item()
    auroc_metric.reset()
    accuracy_metric.reset()
    
    history = [n_concepts, model.val_losses[-1], auc, acc]
    history_multiclass.append(np.array(history))
history_multiclass = np.array(history_multiclass)
history_multiclass.shape


In [ ]:
# plot
plt.plot(history_multiclass[:, 0], history_multiclass[:, 2], label='AUC')
plt.plot(history_multiclass[:, 0], history_multiclass[:, 3], label='ACC')

plt.xlabel('Num Concepts')
plt.ylabel('Criteria')
plt.title('Plot of Concepts vs Criteria')
plt.xticks(np.arange(min(history_multiclass[:, 0]), max(history_multiclass[:, 0])+1, 1))

for x,_y in zip(history_multiclass[:, 0], history_multiclass[:, 2]):
    label = "{:.2f}".format(_y)
    plt.annotate(label, # this is the text
                 (x,_y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
    
for x,_y in zip(history_multiclass[:, 0], history_multiclass[:, 3]):
    label = "{:.2f}".format(_y)
    plt.annotate(label, # this is the text
                 (x,_y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

plt.legend()
plt.show()


In [ ]:
# feature weights
n_concepts = 5

model = initializeModel(n_concepts, input_dim, changing_dim, seq_len, num_classes)
model.fit(train_loader, val_loader, weights, model_path.format(n_concepts), 1000)

for name, param in model.named_parameters():
    if "bottleneck.weight" in name:
        bottleneck_weights = param
feature_weights = bottleneck_weights.cpu().detach().numpy()

feature_weights.shape

In [ ]:
# visualize weight magnitudes
for c in range(n_concepts):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    inds = np.argsort(-np.abs(feature_weights[c]))[:100]
    ax.bar(np.arange(1,101),np.abs(feature_weights[c])[inds])
    ax.set_xlabel("Top 100 features")
    ax.set_ylabel("abs value of feature coefficient")
    plt.show()


In [ ]:
# get 90th percentile of feature weights
sum90p = np.sum(np.abs(feature_weights), axis=-1)*0.90
sum90p.shape


In [ ]:
# get top K indizes
top_k_inds = []
for c in range(n_concepts):
    topkinds_conc = []
    curr_sum = 0
    inds = np.argsort(-np.abs(feature_weights[c])) #desc
    sorted_weights = feature_weights[c][inds]
    
    for ind, weight in zip(inds, sorted_weights):
        curr_sum += abs(weight)
        if curr_sum <= sum90p[c]:
            topkinds_conc.append(ind)
        else:
            break
    
    # if selects less than 10, choose 10 best
    if len(topkinds_conc) < 10:
        topkinds_conc = np.argsort(-np.abs(feature_weights[c]))[:10].tolist()
    
    top_k_inds.append(topkinds_conc)

top_k_inds

In [ ]:
# write top k inds to csv
filename = experiment_folder + "top-k/top_k_inds_c{}.csv".format(n_concepts)

directory = os.path.dirname(filename)
if not os.path.exists(directory):
    os.makedirs(directory)

# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)
    # writing the data rows 
    csvwriter.writerows(top_k_inds)


In [ ]:
data_cols = [i for i in range(1,14)] + [str(i) + "_ind" for i in range(1,14)]

for c, _list in enumerate(top_k_inds):
    for ind in _list:
        name, summary = getConcept(data_cols, input_dim, changing_dim, int(ind))
        print(f"Concept {c}: ID {ind}, Feature {name}, Summary {summary}")


In [ ]:
greedy_results = greedy_selection(auroc_metric, test_loader, top_k_inds, model, track_metrics={"acc": accuracy_metric})
greedy_results.head()

In [ ]:
top_k_csv_file = experiment_folder + "top-k/bottleneck_r{}_c{}_topkinds.csv".format(random_seed, n_concepts)

# writing to csv file
with open(top_k_csv_file, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(greedy_results.columns)
    # writing the data rows 
    for row in greedy_results.itertuples(index=False):
        csvwriter.writerow(list(row))


In [ ]:
data_cols = [i for i in range(1,14)] + [str(i) + "_ind" for i in range(1,14)]

sorted_ = greedy_results.sort_values(["Concept", "ID"])

for row in sorted_.itertuples(index=False):
    name, summary = getConcept(data_cols, input_dim, changing_dim, row[1])
    print(f"Concept {row[2]}: ID {row[1]}, Feature {name}, Summary {summary}")

In [ ]:
plt.plot(greedy_results["Score"])
plt.plot(greedy_results["acc"])
plt.show()

In [ ]:
top_k_csv_file = "/workdir/optimal-summaries-public/_models/arabic/multiclass/top-k/bottleneck_r1_c6_topkinds.csv"
n_concepts = 6
model = initializeModel(n_concepts, input_dim, changing_dim, seq_len, num_classes, top_k=top_k_csv_file)
# model.fit(train_loader, val_loader, weights, model_path.format(n_concepts), 1000)

model.eval()
with torch.no_grad():
    for batch_idx, (Xb, yb) in enumerate(test_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        probs = model.forward_probabilities(Xb)
        
        auc = auroc_metric(probs, yb).item()
        acc = accuracy_metric(probs, yb).item()
    auc = auroc_metric.compute().item()
    acc = accuracy_metric.compute().item()
    auroc_metric.reset()
    accuracy_metric.reset()

print(auc)
print(acc)


In [ ]:
model.fit(train_loader, val_loader, weights, save_model_path="/workdir/optimal-summaries-public/_models/arabic/multiclass/top-k/arabic_c6_finetuned.pt", epochs=3000)

model.eval()
with torch.no_grad():
    for batch_idx, (Xb, yb) in enumerate(test_loader):
        Xb, yb = Xb.to(device), yb.to(device)
        probs = model.forward_probabilities(Xb)
        
        auc = auroc_metric(probs, yb)
        acc = accuracy_metric(probs, yb)
    auc = auroc_metric.compute().item()
    acc = accuracy_metric.compute().item()
    auroc_metric.reset()
    accuracy_metric.reset()
    
print(auc)
print(acc)
